In [1]:
# 设置LangSmith跟踪调试
! export LANGCHAIN_TRACING_V2="true"
# https://smith.langchain.com 设置中 申请api_key
! export LANGCHAIN_API_KEY="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"

In [2]:
import getpass
import os

# https://fireworks.ai/api-keys 申请api_key
os.environ["FIREWORKS_API_KEY"] = "HzeWA5AODgQjkCA4vhJYw2wKeXfyG22Qv1CV86q37l2OjAKg"

from langchain_fireworks import ChatFireworks

llm = ChatFireworks(model="accounts/fireworks/models/mixtral-8x7b-instruct")

In [5]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_fireworks import FireworksEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, 
                                               chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=FireworksEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

"Task Decomposition is a process that breaks down a complex task into smaller, manageable steps. This can be done using a Language Model (LLM) with simple prompting, task-specific instructions, or human inputs. The purpose is to transform big tasks into multiple simpler tasks, providing insights into the model's thinking process. This method is often used in AI systems like autonomous agents for efficient task planning and execution."